# Stationay SCM and DAG (STAT.) from figure 1 in paper

In [7]:
import sys
sys.path.append("../src/")
sys.path.append("..")

from src.examples.example_setups import setup_stat_scm
from src.utils.sequential_causal_functions import sequentially_sample_model

# Models
from src.methods.bo import BO
from src.methods.cbo import CBO
from src.methods.abo import ABO
from src.methods.bo import DCBO


ModuleNotFoundError: No module named 'src.gaussian_process_utils'

## Sample SEM to get observational samples

In [2]:
T = 3 # In this example, as in the paper, we consider three time-slices
init_sem, sem, dag_view, dag  = setup_stat_scm(T=3)

NameError: name 'setup_stat_scm' is not defined

In [ ]:
# Number of independent samples per time-index
observational_samples = sequentially_sample_model(initial_structural_equation_model,
                                                  structural_equation_model,
                                                  total_timesteps=3,
                                                  sample_count=5,
                                                  epsilon=None)